# Benthic Activity Detection V4 - Parameter Optimization

This notebook allows you to:
1. Load multiple test videos
2. Run detection with different parameter combinations **in parallel**
3. Compare results visually and quantitatively
4. Find optimal parameters for your specific videos

In [15]:
# Setup and imports
import sys
from pathlib import Path

# Add cv_scripts to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'cv_scripts'))

import cv2
import numpy as np
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, as_completed
from dataclasses import asdict
from datetime import datetime
import json
import matplotlib.pyplot as plt
from IPython.display import Video, display, HTML
import ipywidgets as widgets
from tqdm.notebook import tqdm
import itertools

from benthic_activity_detection_v4 import (
    DetectionParams, TrackingParams, ValidationParams,
    process_video, detect_blobs, preprocess_frame
)

print("Imports successful!")

ModuleNotFoundError: No module named 'cv2'

## 1. Configure Test Videos

Add paths to your test videos below. These should be **motion videos** (background-subtracted).

In [2]:
# Configure your test videos here
TEST_VIDEOS = [
    # Add your video paths here
    # r"C:\path\to\video1_motion.mp4",
    # r"C:\path\to\video2_motion.mp4",
]

# Or auto-discover motion videos in a directory
VIDEO_SEARCH_DIR = project_root / "cv_output"

if not TEST_VIDEOS and VIDEO_SEARCH_DIR.exists():
    TEST_VIDEOS = list(VIDEO_SEARCH_DIR.glob("*_motion.mp4"))[:5]  # Limit to 5 videos
    print(f"Auto-discovered {len(TEST_VIDEOS)} motion videos:")
    for v in TEST_VIDEOS:
        print(f"  - {v.name}")
else:
    print(f"Using {len(TEST_VIDEOS)} configured videos")

# Output directory for results
OUTPUT_DIR = project_root / "cv_output" / "parameter_tuning"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print(f"\nOutput directory: {OUTPUT_DIR}")

Using 0 configured videos

Output directory: C:\Users\Lorenz\DataApp\cv_output\parameter_tuning


## 2. Define Parameter Grid

Configure the parameter combinations you want to test.

In [3]:
# Parameter grid for testing
# Each list contains values to try for that parameter

PARAM_GRID = {
    # Detection thresholds
    'dark_threshold': [10, 15, 18, 25],      # Shadow sensitivity
    'bright_threshold': [30, 40, 50],        # Reflection sensitivity
    'min_area': [50, 75, 100],               # Minimum blob size
    
    # Coupling parameters  
    'coupling_distance': [80, 100, 120],     # Max distance for shadow-reflection pairing
    
    # Tracking parameters
    'max_skip_frames': [60, 90, 120],        # Frames to track through rest periods
}

# Fixed parameters (not varied during grid search)
FIXED_PARAMS = {
    'threshold': 30,
    'max_area': 2000,
    'min_circularity': 0.3,
    'max_aspect_ratio': 3.0,
    'morph_kernel_size': 5,
    'require_coupling': False,
    'coupling_boost': 1.3,
    'max_distance': 75.0,
    'rest_zone_radius': 120,
    'min_track_length': 4,
    'min_displacement': 8.0,
    'max_speed': 30.0,
    'min_speed': 0.1,
}

# Calculate total combinations
total_combinations = 1
for values in PARAM_GRID.values():
    total_combinations *= len(values)

print(f"Parameter grid will test {total_combinations} combinations")
print(f"With {len(TEST_VIDEOS)} videos = {total_combinations * len(TEST_VIDEOS)} total runs")

Parameter grid will test 324 combinations
With 0 videos = 0 total runs


In [4]:
# Generate all parameter combinations
def generate_param_combinations(grid: dict) -> list:
    """Generate all combinations from parameter grid."""
    keys = list(grid.keys())
    values = list(grid.values())
    
    combinations = []
    for combo in itertools.product(*values):
        param_dict = dict(zip(keys, combo))
        combinations.append(param_dict)
    
    return combinations

param_combinations = generate_param_combinations(PARAM_GRID)
print(f"Generated {len(param_combinations)} parameter combinations")
print(f"\nFirst 3 combinations:")
for i, combo in enumerate(param_combinations[:3]):
    print(f"  {i+1}: {combo}")

NameError: name 'itertools' is not defined

## 3. Quick Preview - Single Frame Analysis

Before running full video processing, preview detection on a single frame to quickly evaluate parameters.

In [5]:
def analyze_single_frame(video_path: Path, frame_idx: int, params: DetectionParams):
    """Analyze a single frame and return detection results with visualization."""
    cap = cv2.VideoCapture(str(video_path))
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ret, frame = cap.read()
    cap.release()
    
    if not ret:
        return None, None, {}
    
    gray = preprocess_frame(frame)
    blobs = detect_blobs(gray, frame_idx, params)
    
    # Draw detections on frame
    annotated = frame.copy()
    
    colors = {
        'coupled': (0, 255, 0),    # Green - high confidence
        'dark': (255, 165, 0),     # Orange - shadow only
        'bright': (0, 255, 255),   # Yellow - reflection only
        'standard': (255, 0, 255)  # Magenta - standard motion
    }
    
    stats = {'coupled': 0, 'dark': 0, 'bright': 0, 'standard': 0}
    
    for blob in blobs:
        x, y, w, h = blob.bbox
        color = colors.get(blob.blob_type, (255, 255, 255))
        cv2.rectangle(annotated, (x, y), (x+w, y+h), color, 2)
        cv2.circle(annotated, (int(blob.centroid[0]), int(blob.centroid[1])), 3, color, -1)
        stats[blob.blob_type] = stats.get(blob.blob_type, 0) + 1
    
    # Convert BGR to RGB for matplotlib
    annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    
    return annotated_rgb, blobs, stats


def compare_params_single_frame(video_path: Path, frame_idx: int, param_list: list):
    """Compare multiple parameter sets on a single frame."""
    n_params = len(param_list)
    cols = min(3, n_params)
    rows = (n_params + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(6*cols, 5*rows))
    if n_params == 1:
        axes = [axes]
    else:
        axes = axes.flatten()
    
    for i, varied_params in enumerate(param_list):
        # Merge with fixed params
        full_params = {**FIXED_PARAMS, **varied_params}
        
        detection_params = DetectionParams(
            threshold=full_params['threshold'],
            dark_threshold=full_params['dark_threshold'],
            bright_threshold=full_params['bright_threshold'],
            min_area=full_params['min_area'],
            max_area=full_params['max_area'],
            min_circularity=full_params['min_circularity'],
            max_aspect_ratio=full_params['max_aspect_ratio'],
            morph_kernel_size=full_params['morph_kernel_size'],
            coupling_distance=full_params.get('coupling_distance', 100),
            require_coupling=full_params['require_coupling'],
            coupling_boost=full_params['coupling_boost']
        )
        
        annotated, blobs, stats = analyze_single_frame(video_path, frame_idx, detection_params)
        
        if annotated is not None:
            axes[i].imshow(annotated)
            title = f"dark={varied_params.get('dark_threshold', 'def')} bright={varied_params.get('bright_threshold', 'def')}\n"
            title += f"min_area={varied_params.get('min_area', 'def')} coupling={varied_params.get('coupling_distance', 'def')}\n"
            title += f"Detections: {len(blobs)} (C:{stats['coupled']} D:{stats['dark']} B:{stats['bright']} S:{stats['standard']})"
            axes[i].set_title(title, fontsize=9)
        axes[i].axis('off')
    
    # Hide unused axes
    for j in range(i+1, len(axes)):
        axes[j].axis('off')
    
    plt.tight_layout()
    plt.show()

print("Single frame analysis functions ready!")

NameError: name 'DetectionParams' is not defined

In [6]:
# Quick test: Compare a few parameter sets on frame 100 of first video
if TEST_VIDEOS:
    test_video = Path(TEST_VIDEOS[0])
    
    # Select a few diverse parameter combinations to compare
    test_params = [
        {'dark_threshold': 10, 'bright_threshold': 30, 'min_area': 50, 'coupling_distance': 100},
        {'dark_threshold': 18, 'bright_threshold': 40, 'min_area': 75, 'coupling_distance': 100},
        {'dark_threshold': 25, 'bright_threshold': 50, 'min_area': 100, 'coupling_distance': 100},
    ]
    
    print(f"Comparing parameters on frame 100 of {test_video.name}")
    compare_params_single_frame(test_video, 100, test_params)
else:
    print("No test videos configured. Add videos to TEST_VIDEOS list.")

No test videos configured. Add videos to TEST_VIDEOS list.


## 4. Parallel Full Video Processing

Run the complete detection pipeline on multiple videos with different parameters **in parallel**.

In [7]:
def run_single_experiment(args: tuple) -> dict:
    """
    Run a single video processing experiment.
    Args is a tuple of (video_path, varied_params, fixed_params, output_dir, experiment_id)
    """
    video_path, varied_params, fixed_params, output_dir, experiment_id = args
    
    # Merge parameters
    full_params = {**fixed_params, **varied_params}
    
    # Create param objects
    detection_params = DetectionParams(
        threshold=full_params['threshold'],
        dark_threshold=full_params['dark_threshold'],
        bright_threshold=full_params['bright_threshold'],
        min_area=full_params['min_area'],
        max_area=full_params['max_area'],
        min_circularity=full_params['min_circularity'],
        max_aspect_ratio=full_params['max_aspect_ratio'],
        morph_kernel_size=full_params['morph_kernel_size'],
        coupling_distance=full_params.get('coupling_distance', 100),
        require_coupling=full_params['require_coupling'],
        coupling_boost=full_params['coupling_boost']
    )
    
    tracking_params = TrackingParams(
        max_distance=full_params['max_distance'],
        max_skip_frames=full_params.get('max_skip_frames', 90),
        rest_zone_radius=full_params['rest_zone_radius']
    )
    
    validation_params = ValidationParams(
        min_track_length=full_params['min_track_length'],
        min_displacement=full_params['min_displacement'],
        max_speed=full_params['max_speed'],
        min_speed=full_params['min_speed']
    )
    
    # Create experiment-specific output dir
    exp_output = Path(output_dir) / f"exp_{experiment_id}"
    exp_output.mkdir(parents=True, exist_ok=True)
    
    try:
        results = process_video(
            Path(video_path),
            exp_output,
            detection_params,
            tracking_params,
            validation_params,
            video_id=Path(video_path).stem,
            run_id=experiment_id
        )
        
        return {
            'experiment_id': experiment_id,
            'video': Path(video_path).name,
            'params': varied_params,
            'status': 'success',
            'valid_tracks': results['summary']['valid_tracks'],
            'total_tracks': results['summary']['total_tracks'],
            'total_detections': results['summary']['total_detections'],
            'coupling_rate': results['summary']['overall_coupling_rate'],
            'processing_time': results['summary']['processing_time'],
            'output_video': results['output_paths']['annotated_video'],
            'output_json': results['output_paths']['results_json'],
        }
        
    except Exception as e:
        return {
            'experiment_id': experiment_id,
            'video': Path(video_path).name,
            'params': varied_params,
            'status': 'error',
            'error': str(e)
        }

print("Parallel processing function ready!")

Parallel processing function ready!


In [8]:
def run_parallel_experiments(
    videos: list,
    param_combinations: list,
    fixed_params: dict,
    output_dir: Path,
    max_workers: int = 4
) -> pd.DataFrame:
    """
    Run all experiments in parallel.
    
    Args:
        videos: List of video paths
        param_combinations: List of parameter dicts to try
        fixed_params: Parameters that don't vary
        output_dir: Base output directory
        max_workers: Number of parallel processes
    
    Returns:
        DataFrame with all experiment results
    """
    # Generate all experiment configurations
    experiments = []
    exp_id = 0
    
    for video in videos:
        for params in param_combinations:
            experiments.append((
                str(video),
                params,
                fixed_params,
                str(output_dir),
                f"{exp_id:04d}"
            ))
            exp_id += 1
    
    print(f"Running {len(experiments)} experiments with {max_workers} workers...")
    print(f"Estimated time: ~{len(experiments) * 30 / max_workers / 60:.1f} minutes (assuming 30s per video)")
    
    results = []
    
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(run_single_experiment, exp): exp for exp in experiments}
        
        with tqdm(total=len(experiments), desc="Processing") as pbar:
            for future in as_completed(futures):
                result = future.result()
                results.append(result)
                
                # Update progress bar with status
                status = "OK" if result['status'] == 'success' else "ERR"
                tracks = result.get('valid_tracks', 0)
                pbar.set_postfix({'last': f"{status} ({tracks} tracks)"})
                pbar.update(1)
    
    # Convert to DataFrame
    df = pd.DataFrame(results)
    
    # Expand params dict into columns
    if 'params' in df.columns:
        params_df = pd.json_normalize(df['params'])
        params_df.columns = [f'param_{c}' for c in params_df.columns]
        df = pd.concat([df.drop('params', axis=1), params_df], axis=1)
    
    return df

print("Parallel experiment runner ready!")

NameError: name 'pd' is not defined

In [9]:
# RUN PARALLEL EXPERIMENTS
# Adjust max_workers based on your CPU cores (recommended: number of cores - 1)

MAX_WORKERS = 4  # Adjust based on your system

if TEST_VIDEOS and param_combinations:
    results_df = run_parallel_experiments(
        videos=TEST_VIDEOS,
        param_combinations=param_combinations,
        fixed_params=FIXED_PARAMS,
        output_dir=OUTPUT_DIR,
        max_workers=MAX_WORKERS
    )
    
    # Save results
    results_path = OUTPUT_DIR / f"experiment_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    results_df.to_csv(results_path, index=False)
    print(f"\nResults saved to: {results_path}")
else:
    print("No videos or parameters configured. Skipping parallel run.")
    results_df = None

No videos or parameters configured. Skipping parallel run.


## 5. Analyze Results

Compare experiment outcomes to find optimal parameters.

In [ ]:
# Display results summary
if results_df is not None and len(results_df) > 0:
    print("=" * 80)
    print("EXPERIMENT RESULTS SUMMARY")
    print("=" * 80)
    
    successful = results_df[results_df['status'] == 'success']
    print(f"\nSuccessful experiments: {len(successful)}/{len(results_df)}")
    
    if len(successful) > 0:
        print(f"\nTop 10 by valid tracks:")
        top_tracks = successful.nlargest(10, 'valid_tracks')[[
            'experiment_id', 'video', 'valid_tracks', 'total_detections', 
            'coupling_rate', 'processing_time'
        ] + [c for c in successful.columns if c.startswith('param_')]]
        display(top_tracks)
        
        print(f"\nTop 10 by coupling rate:")
        top_coupling = successful.nlargest(10, 'coupling_rate')[[
            'experiment_id', 'video', 'valid_tracks', 'coupling_rate'
        ] + [c for c in successful.columns if c.startswith('param_')]]
        display(top_coupling)
else:
    print("No results to analyze. Run experiments first.")

In [ ]:
# Visualize parameter effects
if results_df is not None and len(results_df) > 0:
    successful = results_df[results_df['status'] == 'success']
    
    if len(successful) > 0:
        param_cols = [c for c in successful.columns if c.startswith('param_')]
        
        fig, axes = plt.subplots(2, len(param_cols), figsize=(5*len(param_cols), 8))
        
        for i, param in enumerate(param_cols):
            # Valid tracks vs parameter
            axes[0, i].scatter(successful[param], successful['valid_tracks'], alpha=0.6)
            axes[0, i].set_xlabel(param.replace('param_', ''))
            axes[0, i].set_ylabel('Valid Tracks')
            axes[0, i].set_title(f'Tracks vs {param.replace("param_", "")}')
            
            # Coupling rate vs parameter
            axes[1, i].scatter(successful[param], successful['coupling_rate'], alpha=0.6, color='orange')
            axes[1, i].set_xlabel(param.replace('param_', ''))
            axes[1, i].set_ylabel('Coupling Rate (%)')
            axes[1, i].set_title(f'Coupling vs {param.replace("param_", "")}')
        
        plt.tight_layout()
        plt.savefig(OUTPUT_DIR / 'parameter_effects.png', dpi=150)
        plt.show()
        print(f"Saved plot to: {OUTPUT_DIR / 'parameter_effects.png'}")

In [ ]:
# Find optimal parameters (aggregated across all videos)
if results_df is not None and len(results_df) > 0:
    successful = results_df[results_df['status'] == 'success']
    
    if len(successful) > 0:
        param_cols = [c for c in successful.columns if c.startswith('param_')]
        
        # Group by parameters, average across videos
        grouped = successful.groupby(param_cols).agg({
            'valid_tracks': 'mean',
            'total_detections': 'mean',
            'coupling_rate': 'mean',
            'processing_time': 'mean',
            'experiment_id': 'count'
        }).rename(columns={'experiment_id': 'video_count'}).reset_index()
        
        print("\nBest parameters by average valid tracks:")
        best = grouped.nlargest(5, 'valid_tracks')
        display(best)
        
        print("\nRecommended optimal parameters:")
        optimal = grouped.loc[grouped['valid_tracks'].idxmax()]
        for col in param_cols:
            print(f"  {col.replace('param_', '')}: {optimal[col]}")

## 6. Review Individual Experiment Videos

Watch annotated videos from specific experiments.

In [ ]:
# Interactive video viewer
def show_experiment_video(experiment_id: str):
    """Display the annotated video from a specific experiment."""
    if results_df is None:
        print("No results available.")
        return
    
    exp_row = results_df[results_df['experiment_id'] == experiment_id]
    if len(exp_row) == 0:
        print(f"Experiment {experiment_id} not found.")
        return
    
    exp_row = exp_row.iloc[0]
    
    if exp_row['status'] != 'success':
        print(f"Experiment {experiment_id} failed: {exp_row.get('error', 'Unknown error')}")
        return
    
    video_path = exp_row['output_video']
    
    print(f"Experiment: {experiment_id}")
    print(f"Video: {exp_row['video']}")
    print(f"Valid tracks: {exp_row['valid_tracks']}")
    print(f"Coupling rate: {exp_row['coupling_rate']:.1f}%")
    
    param_cols = [c for c in exp_row.index if c.startswith('param_')]
    print("Parameters:")
    for col in param_cols:
        print(f"  {col.replace('param_', '')}: {exp_row[col]}")
    
    if Path(video_path).exists():
        display(Video(video_path, width=800))
    else:
        print(f"Video file not found: {video_path}")

# Example: View best experiment
if results_df is not None and len(results_df) > 0:
    successful = results_df[results_df['status'] == 'success']
    if len(successful) > 0:
        best_exp = successful.loc[successful['valid_tracks'].idxmax(), 'experiment_id']
        print(f"Best experiment ID: {best_exp}")
        # Uncomment to view:
        # show_experiment_video(best_exp)

In [ ]:
# Interactive experiment selector
if results_df is not None and len(results_df) > 0:
    exp_ids = results_df['experiment_id'].tolist()
    
    dropdown = widgets.Dropdown(
        options=exp_ids,
        description='Experiment:',
        style={'description_width': 'initial'}
    )
    
    button = widgets.Button(description='View Video')
    output = widgets.Output()
    
    def on_button_click(b):
        with output:
            output.clear_output()
            show_experiment_video(dropdown.value)
    
    button.on_click(on_button_click)
    
    display(widgets.HBox([dropdown, button]))
    display(output)

## 7. Export Optimal Configuration

Save the best parameters for use in production.

In [ ]:
def export_optimal_config(results_df: pd.DataFrame, output_path: Path, metric: str = 'valid_tracks'):
    """Export the optimal parameters as a JSON config file."""
    successful = results_df[results_df['status'] == 'success']
    
    if len(successful) == 0:
        print("No successful experiments to export.")
        return
    
    param_cols = [c for c in successful.columns if c.startswith('param_')]
    
    # Group and find best
    grouped = successful.groupby(param_cols).agg({
        metric: 'mean'
    }).reset_index()
    
    best_row = grouped.loc[grouped[metric].idxmax()]
    
    # Build config
    optimal_config = {
        'detection_params': {
            **{k: v for k, v in FIXED_PARAMS.items() if k in [
                'threshold', 'max_area', 'min_circularity', 'max_aspect_ratio',
                'morph_kernel_size', 'require_coupling', 'coupling_boost'
            ]},
            **{col.replace('param_', ''): best_row[col] for col in param_cols 
               if col.replace('param_', '') in ['dark_threshold', 'bright_threshold', 'min_area', 'coupling_distance']}
        },
        'tracking_params': {
            'max_distance': FIXED_PARAMS['max_distance'],
            'rest_zone_radius': FIXED_PARAMS['rest_zone_radius'],
            **{col.replace('param_', ''): best_row[col] for col in param_cols 
               if col.replace('param_', '') in ['max_skip_frames']}
        },
        'validation_params': {
            'min_track_length': FIXED_PARAMS['min_track_length'],
            'min_displacement': FIXED_PARAMS['min_displacement'],
            'max_speed': FIXED_PARAMS['max_speed'],
            'min_speed': FIXED_PARAMS['min_speed'],
        },
        'metadata': {
            'optimized_for': metric,
            'best_value': float(best_row[metric]),
            'timestamp': datetime.now().isoformat(),
            'experiments_count': len(successful)
        }
    }
    
    # Convert numpy types
    def convert_types(obj):
        if isinstance(obj, (np.integer, np.int64, np.int32)):
            return int(obj)
        elif isinstance(obj, (np.floating, np.float64, np.float32)):
            return float(obj)
        elif isinstance(obj, dict):
            return {k: convert_types(v) for k, v in obj.items()}
        return obj
    
    optimal_config = convert_types(optimal_config)
    
    with open(output_path, 'w') as f:
        json.dump(optimal_config, f, indent=2)
    
    print(f"Optimal configuration saved to: {output_path}")
    print(f"\nOptimal parameters (optimized for {metric}):")
    print(json.dumps(optimal_config, indent=2))
    
    return optimal_config

# Export optimal config
if results_df is not None and len(results_df) > 0:
    config_path = OUTPUT_DIR / 'optimal_config.json'
    optimal = export_optimal_config(results_df, config_path, metric='valid_tracks')

## 8. Quick Single Video Test

Test specific parameters on a single video without the full grid search.

In [ ]:
# Quick test with specific parameters
def quick_test(
    video_path: str,
    dark_threshold: int = 18,
    bright_threshold: int = 40,
    min_area: int = 75,
    coupling_distance: int = 100,
    max_skip_frames: int = 90
):
    """Run a quick test with specific parameters."""
    
    detection_params = DetectionParams(
        threshold=30,
        dark_threshold=dark_threshold,
        bright_threshold=bright_threshold,
        min_area=min_area,
        max_area=2000,
        min_circularity=0.3,
        max_aspect_ratio=3.0,
        morph_kernel_size=5,
        coupling_distance=coupling_distance,
        require_coupling=False,
        coupling_boost=1.3
    )
    
    tracking_params = TrackingParams(
        max_distance=75.0,
        max_skip_frames=max_skip_frames,
        rest_zone_radius=120
    )
    
    validation_params = ValidationParams(
        min_track_length=4,
        min_displacement=8.0,
        max_speed=30.0,
        min_speed=0.1
    )
    
    output_dir = OUTPUT_DIR / "quick_test"
    output_dir.mkdir(parents=True, exist_ok=True)
    
    results = process_video(
        Path(video_path),
        output_dir,
        detection_params,
        tracking_params,
        validation_params
    )
    
    print(f"\nOutput video: {results['output_paths']['annotated_video']}")
    return results

# Example usage:
# results = quick_test(r"C:\path\to\video_motion.mp4", dark_threshold=15, min_area=50)

In [ ]:
# Interactive parameter tester
if TEST_VIDEOS:
    video_dropdown = widgets.Dropdown(
        options=[(Path(v).name, str(v)) for v in TEST_VIDEOS],
        description='Video:',
        style={'description_width': 'initial'}
    )
    
    dark_slider = widgets.IntSlider(value=18, min=5, max=40, step=1, description='Dark threshold:')
    bright_slider = widgets.IntSlider(value=40, min=20, max=80, step=5, description='Bright threshold:')
    area_slider = widgets.IntSlider(value=75, min=25, max=200, step=25, description='Min area:')
    coupling_slider = widgets.IntSlider(value=100, min=50, max=150, step=10, description='Coupling dist:')
    skip_slider = widgets.IntSlider(value=90, min=30, max=150, step=15, description='Max skip frames:')
    
    run_button = widgets.Button(description='Run Test', button_style='primary')
    output = widgets.Output()
    
    def on_run_click(b):
        with output:
            output.clear_output()
            print("Running test...")
            results = quick_test(
                video_dropdown.value,
                dark_threshold=dark_slider.value,
                bright_threshold=bright_slider.value,
                min_area=area_slider.value,
                coupling_distance=coupling_slider.value,
                max_skip_frames=skip_slider.value
            )
            print(f"\nValid tracks: {results['summary']['valid_tracks']}")
            print(f"Coupling rate: {results['summary']['overall_coupling_rate']:.1f}%")
    
    run_button.on_click(on_run_click)
    
    display(widgets.VBox([
        video_dropdown,
        dark_slider,
        bright_slider,
        area_slider,
        coupling_slider,
        skip_slider,
        run_button,
        output
    ]))
else:
    print("Add videos to TEST_VIDEOS to use the interactive tester.")